# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List

In [2]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

def bagging(X, y):
    ids = np.arange(X.shape[0])
    ids_in_bag = np.random.choice(ids, size=(X.shape[0]))
    ids_oob = ids[~np.in1d(ids, ids_in_bag)]
    return (X[ids_in_bag], y[ids_in_bag]), (X[ids_oob], y[ids_oob])

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
class DecisionTreeLeaf:
    """
    Attributes
    ----------
    y : dict
        Словарь, отображающий метки в вероятность того, что объект, попавший в данный лист, принадлежит классу, соответствующиему метке 
    """
    def __init__(self, labels):
        ulabels, cnts = np.unique(labels, return_counts=True)
        self.y = ulabels[np.argmax(cnts)]
        cnts = np.array(cnts, dtype=np.float)
        cnts /= labels.shape[0]

        self.dict = {label: prob for label, prob in zip(ulabels, cnts)}


class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value. 
    """
    def __init__(self, split_dim: int, split_value: float, 
                 left: Union['DecisionTreeNode', DecisionTreeLeaf], 
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

In [4]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.criterion = gini if criterion == "gini" else entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = np.floor(np.sqrt(X.shape[1])).astype(np.int) if max_features == "auto" else max_features

        self.bag, self.oob = bagging(X, y)

    def _step(self, X, y, depth: int = 0):
        if self.max_depth is not None and depth >= self.max_depth:
            return DecisionTreeLeaf(y)
        is_leaf = True
        gain2max = 0
        for feature in np.random.choice(X.shape[1], size=self.max_features, replace=False):
            potential_values = np.unique(X[:, feature])
            for value in potential_values:
                left = X[:, feature] == 0
                right = ~left
                left_node_size = np.sum(left)
                right_node_size = X.shape[0] - left_node_size
                cur_gain = gain(y[left], y[right], self.criterion)

                if cur_gain >= gain2max and left_node_size >= self.min_samples_leaf and right_node_size >= self.min_samples_leaf:
                    is_leaf = False
                    gain2max = cur_gain
                    feature2split = feature
                    value2split = value
                    left_node = left.copy()
                    right_node = right.copy()
                    
        return DecisionTreeLeaf(y) if is_leaf else DecisionTreeNode(feature2split, 
                                                                    value2split, 
                                                                    self._step(X[left_node], y[left_node], depth + 1), 
                                                                    self._step(X[right_node], y[right_node], depth + 1))


    def fit(self):
        X, y = self.bag
        self.root = self._step(X, y)

    def _split(self, x, node: DecisionTreeNode):
        val = node.split_value
        idx = node.split_dim

        if x[idx] < val:
            return node.left
        else:
            return node.right

    def dive(self, x, node):
        if isinstance(node, DecisionTreeLeaf):
            return node.dict
        return self.dive(x, self._split(x, node))

    def get_acc(self, X, y):
        return np.sum(self.predict(X) != y) / y.shape[0]

    def get_error(self, X, y):
        return 1 - self.get_acc(X, y)

    def predict_proba(self, X: np.ndarray) ->  List[Dict[Any, float]]:
        """
        Предсказывает вероятность классов для элементов из X.

        Parameters
        ----------
        X : np.ndarray
            Элементы для предсказания.
        
        Return
        ------
        List[Dict[Any, float]]
            Для каждого элемента из X возвращает словарь 
            {метка класса -> вероятность класса}.
        """
        out = []
        for x in X:
            out.append(self.dive(x, self.root))
        return out

    def predict(self, X):
        return np.array([max(p.keys(), key=lambda k: p[k]) for p in self.predict_proba(X)], dtype=np.int)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [5]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = gini if criterion == "gini" else entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.estsimators = []
    
    def fit(self, X, y):
        for _ in range(self.n_estimators):
            dt_ = DecisionTree(X, y, criterion=self.criterion, 
                               max_depth=self.max_depth, 
                               min_samples_leaf=self.min_samples_leaf, 
                               max_features=self.max_features)
            dt_.fit()
            self.estsimators.append(dt_)
    
    def predict(self, X):
        predictions = np.zeros((self.n_estimators, X.shape[0]), dtype=int)
        for i, dt in enumerate(self.estsimators):
            predictions[i] = dt.predict(X)

        out = np.zeros(X.shape[0])
        for i, pred in enumerate(predictions.T):
            out[i] = np.bincount(pred).argmax()
        return out


    def get_feature_importance(self):
        num_features = self.estsimators[0].oob[0].shape[1]
        out = np.zeros((self.n_estimators, num_features))
        for i, dt in enumerate(self.estsimators):
            oob_acc = dt.get_error(dt.oob[0], dt.oob[1])
            for j in range(num_features):
                shuffled_ids = np.random.permutation(range(dt.oob[0].shape[0]))
                X_ = dt.oob[0].copy()
                X_[:, j] = X_[shuffled_ids, j]

                out[i, j] = oob_acc - dt.get_error(X_, dt.oob[1])
        return np.mean(out, axis=0)

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [6]:
def feature_importance(rfc):
    return rfc.get_feature_importance()

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [7]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [2.94407686e-05 2.20079884e-03 1.69027520e-01 1.69024412e-01
 3.19102449e-01 2.43177305e-03]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [8]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [9]:
X, y_age, y_sex, features = read_dataset("vk.csv")
y_age = preprocessing.LabelEncoder().fit_transform(y_age)
y_sex = preprocessing.LabelEncoder().fit_transform(y_sex)
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [10]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.691046658259773
Most important features:
1. rhymes
2. 4ch
3. styd.pozor
4. ovsyanochan
5. mudakoff
6. dayvinchik
7. pravdashowtop
8. tumblr_vacuum
9. iwantyou
10. rapnewrap
11. pixel_stickers
12. ne1party
13. bot_maxim
14. bestad
15. fuck_humor
16. i_des
17. memeboizz
18. pozor
19. girlmeme
20. bog_memes


#### Пол

In [11]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i + 1) + ".", name)

Accuracy: 0.8638083228247163
Most important features:
1. 40kg
2. zerofat
3. mudakoff
4. modnailru
5. girlmeme
6. be.beauty
7. 9o_6o_9o
8. sh.cook
9. beauty
10. be.women
11. i_d_t
12. reflexia_our_feelings
13. woman.blog
14. femalemem
15. igm
16. thesmolny
17. cook_good
18. rapnewrap
19. 4ch
20. recipes40kg


### CatBoost
В качестве альтернативы попробуем CatBoost. 

Установить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [12]:
X, y = synthetic_dataset(1000)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42)

In [13]:
rfc = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent',
    loss_function='MultiClass'
)

rfc.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
)

In [14]:
print("Accuracy:", np.mean(rfc.predict(X_validation).flatten() == y_validation))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [8.76700807e-03 6.18909182e-03 2.92537392e+01 2.73267033e+01
 4.34027323e+01 1.86915561e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [15]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

In [16]:
rfc = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent',
    loss_function='MultiClass'
)

#### Возраст

In [17]:
rfc.fit(
    X_train, y_age_train,
    eval_set=(X_eval, y_age_eval),
)

In [18]:
print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7528373266078184
Most important features:
1. ovsyanochan
2. styd.pozor
3. 4ch
4. leprum
5. mudakoff
6. dayvinchik
7. rhymes
8. xfilm
9. fuck_humor
10. iwantyou


#### Пол

In [19]:
rfc.fit(
    X_train, y_sex_train,
    eval_set=(X_eval, y_sex_eval),
)

In [20]:
print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8701134930643127
Most important features:
1. 40kg
2. mudakoff
3. modnailru
4. girlmeme
5. igm
6. academyofman
7. 4ch
8. zerofat
9. i_d_t
10. femalemem
